In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Defination and Components of a Time Series Data

Any data that has a time component involved in it is termed as a time-series data. For example, the number of orders made on a product ordering app per day is an example of time-series data

**Different components of time series data**

**Trend**:- As the name suggests trend depicts the variation in the output as time increases.It is often non-linear. Sometimes we will refer to trend as “changing direction” when it might go from an increasing trend to a decreasing trend.

**Level**:- It basically depicts baseline value for the time series.

**Seasonal**:- As its name depicts it shows the repeated pattern over time. In layman terms, it shows the seasonal variation of data over time.

**Noise**:- It is basically external noises that vary the data randomly.

* we want to forecast sales and will follow different approch for that.

# Import Libraries And Load Data

In [ ]:
## import all required libraries
import numpy as np ## for linear algebra
import pandas as pd ## data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt ## for visualisation
import seaborn as sns ## for visualisation
import warnings ## for filterout warnings
warnings.filterwarnings("ignore")

In [ ]:
## load our sales data

sales_train = pd.read_csv('../input/demand-forecasting-kernels-only/train.csv',parse_dates=['date'],index_col=['date']) ## train data
sales_test = pd.read_csv('../input/demand-forecasting-kernels-only/test.csv',parse_dates=['date'],index_col=['date']) ## test data

In [ ]:
## check train head

sales_train.head()

In [ ]:
## check train shape

sales_train.shape

In [ ]:
## check info of train data

sales_train.info()

* check for null values in train and test

In [ ]:
## null check in train

sales_train.isnull().sum()

In [ ]:
## null check for test

sales_test.isnull().sum()

* visualise the sales data and try find some pattern on it.

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(sales_train['sales'].resample('W').sum(),label="sales")
plt.show()

* As we can see sales have  seasonal and upword trend.

In [ ]:
## create dataset for every shop and visualize sales trend

shop1 = sales_train[sales_train.store==1]['sales'].sort_index(ascending=True)
shop2 = sales_train[sales_train.store==2]['sales'].sort_index(ascending=True)
shop3 = sales_train[sales_train.store==3]['sales'].sort_index(ascending=True)
shop4 = sales_train[sales_train.store==4]['sales'].sort_index(ascending=True)


In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,figsize=(12,13))
shop1.resample('W').sum().plot(ax=ax1)
shop2.resample('W').sum().plot(ax=ax2)
shop3.resample('W').sum().plot(ax=ax3)
shop4.resample('W').sum().plot(ax=ax4)
plt.show()

In [ ]:
import pylab
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
rcParams['figure.figsize'] = 12, 4
decomposition = seasonal_decompose(shop1, model='additive' ,period=365) # additive seasonal index
fig = decomposition.trend.plot()
plt.show()

* trend of shop1 sales

In [ ]:
rcParams['figure.figsize'] = 12, 4
decomposition2 = seasonal_decompose(shop2, model='additive' ,period=365) # additive seasonal index
fig = decomposition2.trend.plot()
plt.show()

* shop2 sales trend

In [ ]:
rcParams['figure.figsize'] = 12, 4
decompositio3 = seasonal_decompose(shop3, model='additive' ,period=365) # additive seasonal index
fig = decompositio3.trend.plot()
plt.show()

* shop3 sales trend

In [ ]:
rcParams['figure.figsize'] = 12, 4
decompositio4 = seasonal_decompose(shop4, model='additive' ,period=365) # additive seasonal index
fig = decompositio4.trend.plot()
plt.show()

* shop4 sales trend.

From the above plots we ca say that 4 shops's sales are showing same kind of seasonal pattern and trend.next we will go deeper and visualise in aspect of store item and more.

In [ ]:
## create one copy of dataframe

df = sales_train.copy()

df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df['dayofweek'] = df.index.dayofweek


* create seperate column containing exact date , month year and day of the week.

# EDA

In [ ]:
## create two pivot tables one for sales of every item per year  

sales_item = pd.pivot_table(df,index=['year'],values='sales',columns=['item'],aggfunc=np.mean).values

## create two pivot tables one for sales of every store per year  

sales_store = pd.pivot_table(df,index=['year'],values='sales',columns=['store'],aggfunc=np.mean).values

## calculate relative sales of items with change in year

## calculate relative sales of shops with change in year

## lets plot relative sales per year with respect to items and store 

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(sales_item/sales_item.mean(0)[np.newaxis])
plt.xlabel('year')
plt.ylabel('relative sales')
plt.title("Relative sales per item for every year")
plt.subplot(122)
plt.plot(sales_store/sales_store.mean(0)[np.newaxis])
plt.xlabel('year')
plt.ylabel('relative sales')
plt.title("Relative sales per store for every year")
plt.show()



* It is clearly visible that there is a clear pattern when we plot relative sales per year with respect to items and store also all items and store is following a kind of simillar patterns.sales are increasing with every year.

In [ ]:
## create two pivot tables one for sales of every item per month  

sales_item = pd.pivot_table(df,index=['month'],values='sales',columns=['item'],aggfunc=np.mean).values

## create two pivot tables one for sales of every store per month  

sales_store = pd.pivot_table(df,index=['month'],values='sales',columns=['store'],aggfunc=np.mean).values

## calculate relative sales of items with change in month

## calculate relative sales of shops with change in month

## lets plot relative sales per month with respect to items and store 

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(sales_item/sales_item.mean(0)[np.newaxis])
plt.xlabel('months')
plt.ylabel('relative sales')
plt.title("Relative sales per item for every month")
plt.subplot(122)
plt.plot(sales_store/sales_store.mean(0)[np.newaxis])
plt.xlabel('month')
plt.ylabel('relative sales')
plt.title("Relative sales per store for every month")
plt.show()



* There is a clear pattern is visible when we plot sales with respect to items and store for every month of the year there is clear upword and downword pattern.enerally sales are more higher on the month of jun and july.

In [ ]:
## create two pivot tables one for sales of every item per day  

sales_item = pd.pivot_table(df,index=['day'],values='sales',columns=['item'],aggfunc=np.mean).values

## create two pivot tables one for sales of every store per month  

sales_store = pd.pivot_table(df,index=['day'],values='sales',columns=['store'],aggfunc=np.mean).values

## calculate relative sales of items with change in day

## calculate relative sales of shops with change in day

## lets plot relative sales per day with respect to items and store 

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(sales_item/sales_item.mean(0)[np.newaxis])
plt.xlabel('day')
plt.ylabel('relative sales')
plt.title("Relative sales per item for every day")
plt.subplot(122)
plt.plot(sales_store/sales_store.mean(0)[np.newaxis])
plt.xlabel('day')
plt.ylabel('relative sales')
plt.title("Relative sales per store for every day")
plt.show()



* There is no clear pattern if we plot sales per store and per items with respect to every month of the year.some of the month's end sales are very low compared to others.

In [ ]:
## create two pivot tables one for sales of every item per dayofweek  

sales_item = pd.pivot_table(df,index=['dayofweek'],values='sales',columns=['item'],aggfunc=np.mean).values

## create two pivot tables one for sales of every store per month  

sales_store = pd.pivot_table(df,index=['dayofweek'],values='sales',columns=['store'],aggfunc=np.mean).values

## calculate relative sales of items with change in dayofweek

## calculate relative sales of shops with change in dayofweek

## lets plot relative sales per dayofweek with respect to items and store 

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(sales_item/sales_item.mean(0)[np.newaxis])
plt.xlabel('day of the week')
plt.ylabel('relative sales')
plt.title("Relative sales per item for every dayofweek")
plt.subplot(122)
plt.plot(sales_store/sales_store.mean(0)[np.newaxis])
plt.xlabel('day of the week')
plt.ylabel('relative sales')
plt.title("Relative sales per store for every dayofweek")
plt.show()



* with each day of the week sales are getting increased it shows a pick on the weekends.

* From the above plots it's clearly visible that every item's sales and every store's sales follows a same kind of pattern.

# Model Building and Evalution.

* we will discuss about different types of models and check their predicton our data.



In [ ]:
## split our train data to test prediction of our model

train = sales_train.iloc[0:730400] ## train 
test = sales_train.iloc[730400:] ## test

# Naive Method

* let's start with the **naive method** where prediction is the last month's sale.

In [ ]:
test_naive = test.copy()
test_naive['forecast'] = train['sales'][len(train)-1]

* create one column containing the forecast now let's calculate their rmse and mape check our prediction.

In [ ]:
import sklearn
from sklearn.metrics import mean_squared_error

score = mean_squared_error(test['sales'],test_naive['forecast'])

results = pd.DataFrame({'model':['naive model'],'mean squared error':round(score,2)})

In [ ]:
## check result

results

# simple average method

* **simple average method** where forecast is the average of all the past sales.

In [ ]:
test_simpleavg = test.copy()
test_simpleavg['forecast'] = train['sales'].mean()

In [ ]:
score_simavg = mean_squared_error(test['sales'],test_simpleavg['forecast']) ## calculate score

results2 = pd.DataFrame({'model':['simple average model'],'mean squared error':round(score_simavg,2)})
results = pd.concat([results,results2])


In [ ]:
results ##check results

* Model is slightly improved

# Simple Moving Average Method

* **simple moving average method** In this method, the forecasts are generally calculated using the average of the time-series data in the moving window considered. The window of the past observations in the time series data keeps moving, and hence the average values keep changing. This helps in forecasting values at every step in the dataset. older sales values have less impact in case of predicting futures sales and recent sales have more impact on future sales.

Let's considered moving window as 12 and check

In [ ]:
#simp_mov_avg = test.copy()
window = 12
simp_mov_avg = train['sales'].rolling(window).mean()

In [ ]:
simp_avgmov = test.copy()
simp_avgmov['forecast'] = simp_mov_avg[-1]

In [ ]:
score_simp_mov_avg = mean_squared_error(test['sales'],simp_avgmov['forecast']) ## calculate score

results3 = pd.DataFrame({'model':['simple average model'],'mean squared error':round(score_simp_mov_avg,2)})
results = pd.concat([results,results3])


In [ ]:
results ## check result

# simple exponential smoothning technique

* **simple exponential smoothning technique** used to forecast the level in the time series data.we start weighting all available observations while exponentially decreasing the weights as we move further back in time to capture the level.

formula = l(t) = a.y(t)+(1-a)l(t-1)

here , l(t) is the level of time period t and forecast of y(t+1) is a function of l(t)

Above formula uses recursively to predict the forecast.

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

model = SimpleExpSmoothing(train['sales'])
model_fit = model.fit(optimized=True)
params = model_fit.params
ses_test = test.copy()
ses_test['forecast'] = model_fit.predict( start='2013-01-01', end='2017-12-31')


In [ ]:
score_ses_avg = mean_squared_error(test['sales'],ses_test['forecast']) ## calculate score

results4 = pd.DataFrame({'model':['simple exponential smoothing  model'],'mean squared error':round(score_ses_avg,2)})
results = pd.concat([results,results4])


In [ ]:
results

# Holt's exponential smoothing technique

* **Holt's exponential smoothing technique** forecasts the level and trend of the time series data. Now the forecast equation is a function of both level and trend.

This method also captures the trend with level to predict or forecast future values

here, y(t+1) = l(t) + b(t)

y(t+1) is the forecast of t+1 time period which is a function of level of 't' time period and trend of 't' time period

l(t) = a*y(t) + (1-a)*(l(t-1)+b(t-1)) we already know that a is the smoothning level parameter

and b(t) = beta*(l(t)-l(t-1)) + (1-beta)*(b(t-1) 'beta' is a smoothning trend parameter also know as smoothning slope in statsmodels package

we can tune this parameter to make a better prediction.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

model_exp = ExponentialSmoothing(np.asarray(train['sales']) ,seasonal_periods=12 ,trend='additive', seasonal=None)
model_fit_exp = model_exp.fit(smoothing_level=0.2, smoothing_slope=0.01, optimized=False)
params = model_fit_exp.params
hes_test = test.copy()
hes_test['forecast'] = model_fit_exp.forecast(182600)


* we can also use optimized set to true to optimize the model for other parameters info check the link : [https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html](http://)

In [ ]:
score_hes_avg = mean_squared_error(test['sales'],hes_test['forecast']) ## calculate score

results5 = pd.DataFrame({'model':['Holts exponential smoothing  model'],'mean squared error':round(score_hes_avg,2)})
results = pd.concat([results,results5])


In [ ]:
results ## check results

# Holt-Winters’ smoothing technique

* The **Holt-Winters’ smoothing technique** forecasts the level, trend as well as the seasonality for a time series data.

This method also captures the seasonality trend and level to predict or forecast future values

here, y(t+1) = l(t) + b(t) + s(t+1-m)

y(t+1) is the forecast of t+1 time period which is a function of level of 't' time period and trend of 't' time period and seasonality of time period 't'

l(t) = a*(y(t)-s(t-m)) + (1-a)*(l(t-1)+b(t-1)) we already know that a is the smoothning level parameter

and b(t) = beta*(l(t)-l(t-1)) + (1-beta)*(b(t-1) 'beta' is a smoothning trend parameter also know as smoothning slope in statsmodels package

and s(t) = gamma(y(t)-b(t-1)-l(t-1)) + (1-gamma)*s(t-m) where 'm' is the number of times a season repeats

'gamma' is smoothning season parameter.

we can tune this parameter to make a better prediction.

There are two types of Holt-winter smoothning technique additive and multiplicative hence we can add season as 'add' for additive model and 'mul' for multiplicative model.

Based upon the forecasting of above models we will chose our final model and test it into our ultimate test data.

